## nomad-parser-nexus demo for Electron Microscopy data (EM)

### **Step -1:** Create a new virtual environment and set up dependencies for jupyterlab_h5web.<br> This section **should only be run if you do not have Jupyter Lab, its extensions and the extra needed packages.** <br>  For use within the Nomad UI this section should be skipped.<br> These cells can be run in a Jupyter notebook or copied to a terminal without the "!".

Run the following to start with a fresh virtualenv in **your terminal** in which you are installing and then running **Jupyter lab** if you wish to: <br>
```pip install virtualenv && virtualenv --python=python3.7 .nexusenv && source .nexusenv/bin/activate```

Install jupyter, jupyter-lab and h5web extensions.

In [ ]:
! pip install --upgrade nodejs && pip install ipywidgets h5py==3.5.0 h5glance==0.7 h5grove==0.0.14 jupyterlab[full]==3.2.9 jupyterlab_h5web[full]==1.3.0 punx==0.2.5 nexpy==0.14.1 silx[full]==1.0.0 && jupyter lab build

Enable the extensions

In [ ]:
! jupyter nbextension enable --py widgetsnbextension

In [ ]:
! jupyter serverextension enable jupyterlab_h5web

### **Step 0:** Installing and testing nomad-parser-nexus module. <br> This section **should only be run if you are not running this within NOMAD.** 

Install nomad and its dependencies. Do not run the following cell if you have a nomad installation running. 

In [ ]:
! pip install --upgrade pip && pip install nomad-lab==1.0.0 --extra-index-url https://gitlab.mpcdf.mpg.de/api/v4/projects/2187/packages/pypi/simple
! pip list

Install the nexusparser and its requirements

In [ ]:
! git clone https://github.com/nomad-coe/nomad-parser-nexus.git --recursive && cd nomad-parser-nexus && git status && pip install -r requirements.txt && pip install -e .[all]

The following cell restarts the kernel after the nexusparser installation

In [ ]:
import os
os._exit(00)

### **Step 1:** Check that everything is ready to go and download example data for APM. 

Check dependencies of nomad and of the nexusparser and prints them. If the nexusparser and nomad-lab are installed, you are ready to go.<br> Check if jupyterlab_h5web server and lab extensions are enabled and OK. 

In [1]:
! pip list | grep 'nomad\|nexus'
! jupyter serverextension list
! jupyter labextension list

nexusformat                   0.7.4
nexusparser                   1.0       /home/mkuehbach/SPRINT-7-ISSUES-110-AND-112/nomad-parser-nexus/parser-nexus
nomad-lab                     1.0.0
config dir: /home/mkuehbach/.jupyter
    jupyterlab_h5web  enabled 
    - Validating...
      jupyterlab_h5web 3.0.0 OK
    jupyter_nbextensions_configurator  enabled 
    - Validating...
Error loading server extension jupyter_nbextensions_configurator
      X is jupyter_nbextensions_configurator importable?
config dir: /home/mkuehbach/SPRINT-7-ISSUES-110-AND-112/nomad-parser-nexus/.pyenv/etc/jupyter
    jupyterlab  enabled 
    - Validating...
      jupyterlab 3.3.2 OK
    jupyterlab_h5web  enabled 
    - Validating...
      jupyterlab_h5web 3.0.0 OK
JupyterLab v3.3.2
/home/mkuehbach/SPRINT-7-ISSUES-110-AND-112/nomad-parser-nexus/.pyenv/share/jupyter/labextensions
        jupyterlab-h5web v3.0.0 enabled OK (python, jupyterlab_h5web)
        @jupyter-widgets/jupyterlab-manager v3.1.0 enabled OK (pytho

For this case, the data can be found in zenodo http://dx.doi.org/10.5281/zenodo.5911409.

In [2]:
# import shutil
# ! cd $PWD && curl --output EM.STEM.Nion.Datasets.3.zip https://zenodo.org/record/5911409/files/EM.STEM.Nion.Datasets.3.zip
# shutil.unpack_archive('EM.STEM.Nion.Datasets.3.zip')
# inspect that the files are there

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  498k  100  498k    0     0   223k      0  0:00:02  0:00:02 --:--:--  223k


### **Step 2:** Run your electron-microscopy-specific dataconverter/readers/em_nion on your example data.

Set the nexusparser directory.

In [3]:
import os
import nexusparser
nexus_dir = os.path.dirname(nexusparser.__file__)  # where the nexusparser module is located!!!!
print(nexus_dir)

Exception while mapping NXcxi_ptycho: type NXtransformations is not supported for transformationsField


/home/mkuehbach/SPRINT-7-ISSUES-110-AND-112/nomad-parser-nexus/parser-nexus/nexusparser


Now we run our parser. The --reader flag takes the electrom microscopy reader (em_nion), the --nxdl flag takes the application definition for this technique.<br> 

Parse the files to convert them to an HDF5 file.

In [19]:
#parser-nexus/tests/data/tools/dataconverter/readers/em/
! python3 {nexus_dir}/tools/dataconverter/convert.py \
--reader em_nion \
--nxdl NXem_nion \
--input-file HAADF_01.npy \
--input-file HAADF_01.ELabFTW.dat \
--input-file HAADF_01.json \
--output em0001.test.nxs

Exception while mapping NXcxi_ptycho: type NXtransformations is not supported for transformationsField
Using em_nion reader to convert the given files:  
• HAADF_01.npy
• HAADF_01.ELabFTW.dat
• HAADF_01.json 
Add metadata which come from other sources...
Extracting data from NionSwift JSON file: HAADF_01.json
Loading: NionSwiftJsonToNexusTranslationTable.ods from...
/home/mkuehbach/SPRINT-7-ISSUES-110-AND-112/nomad-parser-nexus/parser-nexus/tests/data/tools/dataconverter/readers/em/readers/em_nion/utils/
Add metadata/data from numpy array(s) representing scans...
Extracting data from NionSwift NPY file: HAADF_01.npy
Add metadata from e.g.ELN/LIMS dump JSON files...
Extracting data from ELN/LIMS/others JSON file: HAADF_01.ELabFTW.dat
The path, /@default, is being written but has no documentation.
The path, /ENTRY[entry]/@default, is being written but has no documentation.
The path, /ENTRY[entry]/em_lab/@default, is being written but has no documentation.
The path, /ENTRY[entry]/em_lab/h

The key take home message is that the command above-specified triggers the automatic creation of the HDF5 file.<br>
This *.nxs file, is an HDF5 file.

### **Step 3:** Inspect the HDF5/NeXus file em*.test.nxs using H5Web

In [14]:
from jupyterlab_h5web import H5Web

In [15]:
# h5_file_name = 'parser-nexus/tests/data/nexus_test_data/201805_WSe2_arpes.nxs'
h5_file_name = 'em0001.test.nxs'

In [16]:
H5Web(h5_file_name)

<jupyterlab_h5web.widget.H5Web object>

Here is where the general template ends. Continue with filling in the notebook with your own post-processing of this *.nxs file.